<a href="https://colab.research.google.com/github/CHDS-2002/AI_history/blob/main/analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализатор Price-листов

In [ ]:
import os
import re
import csv
from google.colab import drive
from typing import List, Dict, Tuple


class PriceAnalyzer:
    def __init__(self):
        self.data = []

    def load_prices(self, folder_path: str) -> None:
        """Загрузка данных из прайс-листов."""
        for file in os.listdir(folder_path):
            if 'price' not in file.lower():
                continue

            with open(os.path.join(folder_path, file), encoding='utf-8') as f:
                reader = csv.DictReader(f)
                rows = list(reader)

                # Определяем индексы нужных столбцов
                columns = next(iter(rows.values()))
                product_index = self._get_column_index(columns, ['название', 'продукт', 'товар', 'наименование'])
                price_index = self._get_column_index(columns, ['цена', 'розница'])
                weight_index = self._get_column_index(columns, ['фасовка', 'масса', 'вес'])

                if any(index is None for index in [product_index, price_index, weight_index]):
                    print(f"В файле {file} отсутствуют нужные столбцы.")
                    continue

                for row in rows:
                    try:
                        product_name = row[columns[product_index]]
                        price = float(row[columns[price_index]])
                        weight = float(row[columns[weight_index]])

                        self.data.append({
                            'name': product_name,
                            'price': price,
                            'weight': weight,
                            'file': file,
                            'price_per_kg': round(price / weight, 2),
                        })
                    except ValueError:
                        print(f"Ошибка преобразования данных в строке файла {file}.")

    @staticmethod
    def _get_column_index(columns: List[str], possible_names: List[str]) -> int:
        """Поиск индекса нужного столбца."""
        for i, column in enumerate(columns):
            if column.lower() in possible_names:
                return i
        return None

    def find_text(self, search_text: str) -> List[Dict]:
        """Поиск товаров по фрагменту названия."""
        results = []
        for item in self.data:
            if search_text.lower() in item['name'].lower():
                results.append(item)

        # Сортировка по цене за килограмм
        sorted_results = sorted(results, key=lambda x: x['price_per_kg'])

        return sorted_results

    def export_to_html(self, filename: str) -> None:
        """Экспорт данных в HTML-файл."""
        with open(filename, 'w', encoding='utf-8') as f:
            f.write('<html><body><table border="1">')
            f.write('<tr><th>№</th><th>Наименование</th><th>Цена</th><th>Вес</th><th>Файл</th><th>Цена за кг.</th></tr>')

            for i, item in enumerate(self.data):
                f.write(f'<tr><td>{i + 1}</td><td>{item["name"]}</td><td>{item["price"]:.2f}</td>'
                        f'<td>{item["weight"]:.2f}</td><td>{item["file"]}</td><td>{item["price_per_kg"]:.2f}</td></tr>')

            f.write('</table></body></html>')

if __name__ == "__main__":
    analyzer = PriceAnalyzer()
    analyzer.load_prices("prices_folder")  # Указываем путь к папке с прайс-листами

    while True:
        user_input = input("Введите фрагмент названия товара для поиска или 'exit' для завершения: ")
        if user_input.lower() == 'exit':
            print("Работа завершена.")
            break

        results = analyzer.find_text(user_input)

        if len(results) > 0:
            print("\nРезультаты поиска:")
            print(f"{'№':<3}{'Наименование':<30}{'цена':<10}{'вес':<10}{'файл':<20}{'цена за кг.':<15}")
            for i, result in enumerate(results):
                print(f"{i+1:<3}{result['name'][:30]:<30}{result['price']:<10.2f}{result['weight']:<10.2f}"
                      f"{result['file']:<20}{result['price_per_kg']:<15.2f}")
        else:
            print("Товар не найден.")

FileNotFoundError: [Errno 2] No such file or directory: 'prices_folder'

# Загрузка файлов

# Анализатор Price-листов

In [ ]:
import os
import re
import pandas as pd
from google.colab import drive
from typing import List, Dict, Tuple
from scipy.optimize import minimize_scalar

class PriceAnalyzer:
    def __init__(self):
        self.data = []

    def load_prices(self, folder_path: str) -> None:
        """Загрузка данных из прайс-листов."""
        files = [file for file in os.listdir(folder_path) if 'price' in file.lower()]

        for file in files:
            df = pd.read_csv(os.path.join(folder_path, file))

            # Определяем индексы нужных столбцов
            product_columns = ['название', 'продукт', 'товар', 'наименование']
            price_columns = ['цена', 'розница']
            weight_columns = ['фасовка', 'масса', 'вес']

            product_index = self._get_column_index(df.columns, product_columns)
            price_index = self._get_column_index(df.columns, price_columns)
            weight_index = self._get_column_index(df.columns, weight_columns)

            if any([index is None for index in [product_index, price_index, weight_index]]):
                print(f"В файле {file} отсутствуют нужные столбцы.")
                continue

            product_col = df.iloc[:, product_index]
            price_col = df.iloc[:, price_index].astype(float)
            weight_col = df.iloc[:, weight_index].astype(float)

            price_per_kg = price_col / weight_col

            self.data.extend([
                {
                    'name': product,
                    'price': price,
                    'weight': weight,
                    'file': file,
                    'price_per_kg': price_per_kg
                }
                for product, price, weight, price_per_kg in zip(product_col, price_col, weight_col, price_per_kg)
            ])

    @staticmethod
    def _get_column_index(columns: List[str], possible_names: List[str]) -> int:
        """Поиск индекса нужного столбца."""
        for i, column in enumerate(columns):
            if column.lower() in possible_names:
                return i
        return None

    def find_text(self, search_text: str) -> List[Dict]:
        """Поиск товаров по фрагменту названия."""
        results = [
            item for item in self.data
            if search_text.lower() in item['name'].lower()
        ]

        # Оптимизация цены за килограмм
        optimized_results = self._optimize_price_per_kg(results)

        return optimized_results

    def _optimize_price_per_kg(self, items: List[Dict]) -> List[Dict]:
        """Оптимизация цены за килограмм."""
        def objective_function(weight):
            total_cost = sum([item['price'] * weight for item in items])
            total_weight = sum([item['weight'] * weight for item in items])
            return total_cost / total_weight

        res = minimize_scalar(objective_function, bounds=(0, 1), method='bounded')
        optimal_weight = res.x

        optimized_items = [
            {
                'name': item['name'],
                'price': item['price'] * optimal_weight,
                'weight': item['weight'] * optimal_weight,
                'file': item['file'],
                'price_per_kg': item['price_per_kg'] * optimal_weight
            }
            for item in items
        ]

        return sorted(optimized_items, key=lambda x: x['price_per_kg'])

    def export_to_html(self, filename: str) -> None:
        """Экспорт данных в HTML-файл."""
        df = pd.DataFrame(self.data)
        df.to_html(filename, index=False)

if __name__ == "__main__":
    PATH = f"analyzer\\Анализатор price-листов"
    analyzer = PriceAnalyzer()
    analyzer.load_prices(PATH) # Указываем путь к папке с price-листами

    while True:
        user_input = input("Введите фрагмент названия товара для поиска или 'exit' для завершения: ")
        if user_input.lower() == 'exit':
            print("Работа завершена.")
            break

        results = analyzer.find_text(user_input)

        if len(results) > 0:
            print("\nРезультаты поиска:")
            print(f"{'№':<3}{'Наименование':<30}{'цена':<10}{'вес':<10}{'файл':<20}{'цена за кг.':<15}")
            for i, result in enumerate(results):
                print(f"{i+1:<3}{result['name'][:30]:<30}{result['price']:<10.2f}{result['weight']:<10.2f}"
                      f"{result['file']:<20}{result['price_per_kg']:<15.2f}")
        else:
            print("Товар не найден.")

FileNotFoundError: [Errno 2] No such file or directory: 'prices_folder'